## Instalação das dependências do projeto

In [ ]:
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy

## Importação das bibliotecas

In [1]:
import requests
import pandas as pd
import numpy as np

## Extração dos dados básicos dos deputados

In [4]:
def get_politicians_id(url : str) -> object:
    """
    Parâmetro
        - URL é o endereço utilizado para se comunicar com a API e extrair os dados pessoais dos deputados.

    Processamento
        - Requisição do tipo get ao endereço da API para capturar os dados pessoais dos deputados.
        - Transformação da resposta da API para o tipo JSON.
        - Criação de um Dataframe Pandas para estruturar os dados.

    Retorno: 
        - Sucesso, retornará um Dataframe Pandas com os dados pessoais dos deputados.
        - Falha, retornará uma mensagem de erro para o usuário.
    """
    try:
        response = requests.get(url)
        response = response.json()
        dataframe = pd.DataFrame.from_dict(response["dados"])
        return dataframe
    except:
        return f"Error during data extraction from {url}"

## Escrita dos dados básicos dos deputados

In [5]:
# Variável com a URL para realizar a requisição a API.
url_politicians_id = "https://dadosabertos.camara.leg.br/api/v2/deputados/"

# Variável armazena o retorno da função "get_politicians_id" com os dados básicos dos deputados.
data_politicians_id = get_politicians_id(url_politicians_id)

# Escrita dos dados básicos dos deputados.
data_politicians_id.to_csv("_datasets/politicians_id.csv", index=False)

# Mostra os cinco primeiros registros.
data_politicians_id.head(5)

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliobrunini@camara.leg.br
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


## Extração das despesas dos deputados

In [19]:
def get_politicians_expenses(id : list, url : str) -> object:
    """
    Parâmetro
        - ID é uma lista de identificadores dos deputados que foram extraídas na etapa "Extração dos dados básicos dos deputados".
        - URL é o endereço utilizado para se comunicar com a API e extrair os dados de despesas dos deputados.

    Processamento
        - PREVIOUS_URL armazena o texto original da URL para ser utilizado posteriormente.
        - DATAFRAME é criado para armazenar os dados das despesas dos deputados.
        - Para cada deputado, 512, é verificado os seus gastos mensais, 13, totalizando cerca de 6656 requisições para capturar as despesas totais.

    Retorno: 
        - Sucesso, retornará um Dataframe Pandas com as despesas dos deputados.
        - Falha, retornará uma mensagem de erro para o usuário.
    """
    try:
        previous_url = url

        dataframe = pd.DataFrame({
            "id" : [],
            "ano" : [],
            "mes" : [],
            "tipoDespesa" : [],
            "codDocumento" : [],
            "tipoDocumento" : [],
            "codTipoDocumento" : [],
            "dataDocumento" : [],
            "numDocumento" : [],
            "valorDocumento" : [],  
            "urlDocumento" : [],
            "nomeFornecedor" : [],
            "cnpjCpfFornecedor" : [],
            "valorLiquido" : [],
            "valorGlos" : [],
            "numRessarcimento" : [],
            "codLote" : [],
            "parcela" : []
        })

        for i in range(len(id)):
            for j in range(1, 13):
                url = previous_url + str(id[i]) + f"/despesas?ano=2023&mes={str(j)}&itens=100"
                response = requests.get(url)
                response = response.json()
                dataframe_aux = pd.DataFrame.from_dict(response["dados"])
                dataframe_aux.insert(loc=0, column="id", value=str(int(id[i])))
                dataframe = pd.concat([dataframe, dataframe_aux])
        return dataframe
    except:
        return f"Error during data extraction from {url}"

## Escrita das despesas dos deputados

In [24]:
# Variável com a URL para realizar a requisição a API.
url_politicians_expenses = "https://dadosabertos.camara.leg.br/api/v2/deputados/"

# Cria lista com todos os identificadores do deputados.
politicians_id = list(data_politicians_id["id"])

# Variável armazena o retorno da função "get_politicians_expenses" com as despesas dos deputados.
data_politicians_expenses = get_politicians_expenses(politicians_id, url_politicians_expenses)

# Escrita das despesas dos deputados.
data_politicians_expenses.to_csv("_datasets/politicians_expenses.csv", index=False)

# Mostra os cinco primeiros registros.
data_politicians_expenses.head(5)

,id,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlos,numRessarcimento,codLote,parcela,valorGlosa
0,220593.0,2023.0,9.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7610066.0,Nota Fiscal Eletrônica,4.0,2023-09-19,35537,52.98,http://www.camara.leg.br/cota-parlamentar/nota...,3XIS COMERCIO VAREJISTA E ATACADISTA DE PAPELA...,4.299980e+13,52.98,NaN,NaN,1969221.0,0.0,0.0
1,220593.0,2023.0,11.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7657491.0,Nota Fiscal,0.0,2023-11-27,01,67.30,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,1.499558e+13,67.30,NaN,NaN,1996163.0,0.0,0.0
2,220593.0,2023.0,5.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7552621.0,Nota Fiscal,0.0,2023-05-05,11533052023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,1.499558e+13,43.20,NaN,NaN,1938600.0,0.0,0.0
3,220593.0,2023.0,6.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7587236.0,Recibos/Outros,1.0,2023-06-05,11533062023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,1.499558e+13,43.20,NaN,NaN,1957493.0,0.0,0.0
4,220593.0,2023.0,7.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7587239.0,Nota Fiscal,0.0,2023-07-05,11533072023001,43.20,https://www.camara.leg.br/cota-parlamentar/doc...,AGUAS CUIABA S.A,1.499558e+13,43.20,NaN,NaN,1957494.0,0.0,0.0


## Tratando os dados básicos dos deputados

In [25]:
# Leitura do arquivo de dados básicos dos deputados.
politicians_id = pd.read_csv("_datasets/politicians_id.csv")

# Dicionarío para renomear as colunas dos dados básicos dos deputados.
politicians_id_columns = {
    "id" : "politician_id",
    "uri" : "politician_website",
    "nome" : "politician_name",
    "siglaPartido" : "politician_party_name",
    "uriPartido" : "politician_party_website",
    "siglaUf" : "politician_state", 
    "idLegislatura" : "politician_legislature", 
    "urlFoto" : "politician_picture_website",
    "email" : "politician_email"
}

# Renomeia as colunas com o dicionário criado.
politicians_id = politicians_id.rename(columns = politicians_id_columns)

# Transforma os dados de todas as colunas em letra minúscula.
for column in politicians_id.columns:
    politicians_id[column] = politicians_id[column].apply(lambda x : str(x).lower())

# Dicionário com o datatype de cada coluna.
politicians_id_types = {
    "politician_id" : str, 
    "politician_website" : str,
    "politician_name" : str,
    "politician_party_website" : str,
    "politician_state" : str,
    "politician_legislature" : str,
    "politician_picture_website" : str,
    "politician_email" : str
}

# Converte as colunas com o dicionário criado.
politicians_id = politicians_id.astype(politicians_id_types)

# Mostra os cinco primeiros registros.
politicians_id.head(5)

,politician_id,politician_website,politician_name,politician_party_name,politician_party_website,politician_state,politician_legislature,politician_picture_website,politician_email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,abilio brunini,pl,https://dadosabertos.camara.leg.br/api/v2/part...,mt,57,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliobrunini@camara.leg.br
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,acácio favacho,mdb,https://dadosabertos.camara.leg.br/api/v2/part...,ap,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,adail filho,republicanos,https://dadosabertos.camara.leg.br/api/v2/part...,am,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,adilson barroso,pl,https://dadosabertos.camara.leg.br/api/v2/part...,sp,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,adolfo viana,psdb,https://dadosabertos.camara.leg.br/api/v2/part...,ba,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


## Tratando as despesas dos deputados

In [44]:
# Leitura do arquivo de despesas dos deputados.
politicians_expenses = pd.read_csv("_datasets/politicians_expenses.csv")

# Dicionarío para renomear as colunas dos dados de despesas dos deputados.
politicians_expenses_columns = {
    "id" : "politician_id",
    "ano" : "expense_year",
    "mes" : "expense_month",
    "tipoDespesa" : "expense_type",
    "codDocumento" : "document_id",
    "tipoDocumento" : "document_type",
    "codTipoDocumento" : "document_type_id",
    "dataDocumento" : "document_date",
    "numDocumento" : "document_number",
    "valorDocumento" : "document_value",
    "urlDocumento" : "document_url",
    "nomeFornecedor" : "supplier_name",
    "cnpjCpfFornecedor" : "supplier_id",
    "valorLiquido" : "net_value",
    "valorGlos" : "glos_value",
    "numRessarcimento" : "refund_value",
    "codLote" : "batch_id",
    "parcela" : "installment_number",
    "valorGlosa" : "gloss_value"
}
# Renomeia as colunas com o dicionário criado.
politicians_expenses = politicians_expenses.rename(columns = politicians_expenses_columns)

# Transforma os dados de todas as colunas em letra minúscula.
for column in politicians_expenses.columns:
    politicians_expenses[column] = politicians_expenses[column].apply(lambda x : str(x).lower())

# Remove os decimais das colunas "politician_id", "expense_year", "expense_month", ... , "installment_number".
politicians_expenses["politician_id"] = politicians_expenses["politician_id"].apply(lambda x: x.split('.')[0])
politicians_expenses["expense_year"] = politicians_expenses["expense_year"].apply(lambda x: x.split('.')[0])
politicians_expenses["expense_month"] = politicians_expenses["expense_month"].apply(lambda x: x.split('.')[0])
politicians_expenses["document_id"] = politicians_expenses["document_id"].apply(lambda x: x.split('.')[0])
politicians_expenses["document_type_id"] = politicians_expenses["document_type_id"].apply(lambda x: x.split('.')[0])
politicians_expenses["supplier_id"] = politicians_expenses["supplier_id"].apply(lambda x: x.split('.')[0])
politicians_expenses["batch_id"] = politicians_expenses["batch_id"].apply(lambda x: x.split('.')[0])
politicians_expenses["installment_number"] = politicians_expenses["installment_number"].apply(lambda x: x.split('.')[0])

# Dicionário com o datatype de cada coluna.
politicians_expenses_types = {
    "politician_id" : str,
    "expense_year" : str,
    "expense_month" : str,
    "expense_type" : str,
    "document_id" : str,
    "document_type" : str,
    "document_type_id" : str,
    "document_date" : str,
    "document_number" : str,
    "document_value" : str,
    "document_url" : str,
    "supplier_name" : str,
    "supplier_id" : str,
    "net_value" : str,
    "glos_value" : str,
    "refund_value" : str,
    "batch_id" : str,
    "installment_number" : str,
    "gloss_value" : str
}

# Converte as colunas com o dicionário criado.
politicians_expenses = politicians_expenses.astype(politicians_expenses_types)

# Cria um coluna com o ANOMES da emissão do documento.
politicians_expenses["document_month"] = politicians_expenses["document_date"].apply(lambda x: str(x[0:7]))

# Reordena o Dataframe Pandas.
politicians_expenses = politicians_expenses[["politician_id", "expense_year", "expense_month", "expense_type",
       "document_id", "document_type", "document_type_id", "document_date", "document_month",
       "document_number", "document_value", "document_url", "supplier_name",
       "supplier_id", "net_value", "glos_value", "refund_value", "batch_id",
       "installment_number", "gloss_value"]]

# Escreve o arquivo de despesas com os tratamentos realizados.
politicians_expenses.to_csv("_datasets/politicians_expenses.csv", index=False)

# Mostra os cinco primeiros registros.
politicians_expenses.head(5)

,politician_id,expense_year,expense_month,expense_type,document_id,document_type,document_type_id,document_date,document_month,document_number,document_value,document_url,supplier_name,supplier_id,net_value,glos_value,refund_value,batch_id,installment_number,gloss_value
0,220593,2023,9,manutenção de escritório de apoio à atividade ...,7610066,nota fiscal eletrônica,4,2023-09-19,2023-09,35537,52.98,http://www.camara.leg.br/cota-parlamentar/nota...,3xis comercio varejista e atacadista de papela...,42999796000188,52.98,nan,nan,1969221,0,0.0
1,220593,2023,11,manutenção de escritório de apoio à atividade ...,7657491,nota fiscal,0,2023-11-27,2023-11,01,67.3,https://www.camara.leg.br/cota-parlamentar/doc...,aguas cuiaba s.a,14995581000153,67.3,nan,nan,1996163,0,0.0
2,220593,2023,5,manutenção de escritório de apoio à atividade ...,7552621,nota fiscal,0,2023-05-05,2023-05,11533052023001,43.2,https://www.camara.leg.br/cota-parlamentar/doc...,aguas cuiaba s.a,14995581000153,43.2,nan,nan,1938600,0,0.0
3,220593,2023,6,manutenção de escritório de apoio à atividade ...,7587236,recibos/outros,1,2023-06-05,2023-06,11533062023001,43.2,https://www.camara.leg.br/cota-parlamentar/doc...,aguas cuiaba s.a,14995581000153,43.2,nan,nan,1957493,0,0.0
4,220593,2023,7,manutenção de escritório de apoio à atividade ...,7587239,nota fiscal,0,2023-07-05,2023-07,11533072023001,43.2,https://www.camara.leg.br/cota-parlamentar/doc...,aguas cuiaba s.a,14995581000153,43.2,nan,nan,1957494,0,0.0


## Validação dos dados básicos dos deputados
...

## Validação das despesas dos deputados
...